### Importing the libraries

In [ ]:
import os,sys,kfp,zipfile,json
from dkube.sdk import *

### Dkube-SDK

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)

### Create model in dkube with url

In [ ]:
model = DkubeModel(username, name="multimodel")
model.update_model_source(source="git")
model.update_git_details(url="https://github.com/mak-454/dkubeio-examples/blob/multimodel/R/classification/multimodel/models.zip")
api.create_model(model)

### Extract model

In [ ]:

def extract_model():
    model_versions = api.get_model_versions(username,"multimodel")
    model_version = model_versions[0]['version']['uuid'] 
    path_to_zip_file = "/home/"+username+"/model/multimodel/"+model_version+"/data/models.zip"
    model_artifacts = "/home/"+username+"/model/multimodel/"+ model_version +"/data/models/"
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(model_artifacts)
        
extract_model()

### Pipeline Components

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")

client = kfp.Client(existing_token=token)

### Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='multimodel-pl',
    description='sample multimodel pipeline with dkube components'
)
def multimodel_pipeline(token):
    
        serving     = dkube_serving_op(token, model = "multimodel" , device='cpu', serving_image='{"image":"ocdr/inf-multimodel:latest"}')

In [ ]:
client.create_run_from_pipeline_func(multimodel_pipeline, arguments={"token":token})